<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1


    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66


    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-18 10:17:28
-------------------
qualified stocks: 87
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  4.44 L
Current:  1.43 C
-------------------
Today PnL: 43.50 K (0.31%)
Current PnL: -16.43 L (-11.05%)
CY Booked + Current PnL: -5.15 L (-3.47%)
-------------------
Total profit:  2.20 L
Total loss:  -18.63 L
-------------------
Total Booked + Current PnL: 21.83 L (17.94%)
Total Booked PnL: 38.26 L (31.44%)
Curr Year Booked PnL: 11.28 L (7.89%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 82.06 L (57.38%)
Deployed:  1.22 C
Current:  1.43 C
CAGR/XIRR %: 10.0%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,APOLLOHOSP,8285.00,-13.75,56.0,H-LC,10.84,172304.0,16127.0,9959.0,-0.71,10.33,5.78,16.71,22.0,1.62,1.24,29.56,X40N,BTT,HEALTHCARE
77,TTKPRESTIG,770.00,105.34,60.0,M-SC,3.96,88471.0,-12306.0,12395.0,-0.21,-12.21,14.01,0.09,245.0,-0.99,0.64,16.23,OX40N,NTT,DURABLES
57,RELIANCE,1533.00,-11.79,60.0,H-LC,5.19,221052.0,10686.0,18104.0,0.23,5.08,8.19,13.68,37.0,0.59,1.60,22.44,XY25,NTT,REFINERIES
50,NATIONALUM,247.44,-43.66,63.0,H-MC,6.80,112508.0,12004.0,18631.0,-0.46,11.94,16.56,30.49,79.0,0.64,0.81,48.22,MH,ATH,METALS
42,ITC,452.00,-37.50,51.0,H-LC,2.52,201199.0,1061.0,19838.0,0.53,0.53,9.86,10.44,4.0,0.05,1.45,6.12,X40,NTT,FMCG


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,IEX,219.00,-29.69,55.0,H-SC,17.07,201012.0,1058.0,96164.0,-0.21,0.53,47.84,48.63,115.0,0.01,1.45,12.14,XR,NTT,MISC
35,ICICIPRULI,790.00,-20.77,45.0,H-MC,2.23,136267.0,479.0,42270.0,0.48,0.35,31.02,31.48,107.0,0.01,0.98,12.51,X40,ATH,INSURANCE
30,HAVELLS,2069.16,0.06,58.0,H-MC,1.87,248056.0,325.0,74739.0,-0.21,0.13,30.13,30.30,92.0,0.00,1.79,13.44,X40,ATH,ELECTRICAL
52,PGHH,17973.08,-30.68,53.0,H-MC,4.20,200625.0,-195.0,68975.0,-0.07,-0.10,34.38,34.25,80.0,-0.00,1.45,5.10,X40,ATH,FMCG


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-17.03,39.0,H-LC,6.38,298903.0,-16939.0,129604.0,-0.54,-5.36,43.36,35.67,5.0,-0.13,2.16,7.75,X40N,ATH,FMCG
6,ASIANPAINT,3465.66,-13.06,47.0,H-LC,7.83,219551.0,-32217.0,85427.0,0.05,-12.80,38.91,21.13,27.0,-0.38,1.58,18.10,X40,ATH,PAINTS
42,ITC,452.00,-37.50,51.0,H-LC,2.52,201199.0,1061.0,19838.0,0.53,0.53,9.86,10.44,4.0,0.05,1.45,6.12,X40,NTT,FMCG
44,JIOFIN,387.00,0.23,52.0,H-LC,13.22,272233.0,4868.0,60599.0,0.22,1.82,22.26,24.48,48.0,0.08,1.96,57.79,XY24,BTT,FINANCE
80,UNITDSPR,1644.00,-12.66,53.0,H-LC,7.78,229070.0,-5074.0,53694.0,-0.40,-2.17,23.44,20.77,86.0,-0.09,1.65,4.60,X40N,NTT,BREWERIES


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,LTIM,7230.2,0.58,71.0,H-LC,1.28,249390.0,3239.0,75964.0,2.19,1.32,30.46,32.18,16.0,0.04,1.80,37.83,X200,ATH,IT
42,ITC,452.0,-37.50,51.0,H-LC,2.52,201199.0,1061.0,19838.0,0.53,0.53,9.86,10.44,4.0,0.05,1.45,6.12,X40,NTT,FMCG
51,NESTLEIND,1377.0,-8.60,56.0,H-LC,4.38,281221.0,15795.0,41002.0,-0.21,5.95,14.58,21.40,11.0,0.39,2.03,14.20,XY25,NTT,FMCG
57,RELIANCE,1533.0,-11.79,60.0,H-LC,5.19,221052.0,10686.0,18104.0,0.23,5.08,8.19,13.68,37.0,0.59,1.60,22.44,XY25,NTT,REFINERIES
20,CIPLA,1795.0,-19.44,54.0,H-LC,5.79,215158.0,10658.0,30746.0,0.70,5.21,14.29,20.25,10.0,0.35,1.55,14.40,X40N,BTT,PHARMA


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,ATULAUTO,844.00,3991.89,66.0,M-SC,5.77,127523.0,-19845.0,71668.0,3.19,-13.47,56.20,35.16,236.0,-0.28,0.92,30.25,XY24,NTT,AUTO
47,LAOPALA,464.00,118.16,55.0,H-SC,2.34,99861.0,-31244.0,82495.0,2.61,-23.83,82.61,39.09,142.0,-0.38,0.72,28.61,AR,NTT,CERAMICS
48,LTIM,7230.20,0.58,71.0,H-LC,1.28,249390.0,3239.0,75964.0,2.19,1.32,30.46,32.18,16.0,0.04,1.80,37.83,X200,ATH,IT
11,BANDHANBNK,400.00,-10.58,53.0,H-SC,5.13,224664.0,-53895.0,311339.0,2.06,-19.35,138.58,92.42,151.0,-0.17,1.62,29.84,XY24,NTT,BANKS
16,BSOFT,836.99,-12.08,58.0,H-SC,15.02,98940.0,-41713.0,114493.0,1.84,-29.66,115.72,51.74,131.0,-0.36,0.71,12.83,XR,ATH,IT


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,HINDZINC,730.22,20.84,58.0,M-LC,1.35,197142.0,-7934.0,120513.0,-1.36,-3.87,61.13,54.89,52.0,-0.07,1.42,19.29,X5K,ATH,METALS
27,GREENPANEL,537.00,235.92,64.0,M-SC,2.40,149889.0,-33189.0,112162.0,-0.90,-18.13,74.83,43.14,230.0,-0.30,1.08,40.99,XY24,NTT,MISC
3,ALKYLAMINE,4546.37,-4.00,48.0,H-SC,12.75,89298.0,-11665.0,106193.0,-0.82,-11.55,118.92,93.63,148.0,-0.11,0.64,33.01,SR,ATH,CHEMICALS
5,APOLLOHOSP,8285.00,-13.75,56.0,H-LC,10.84,172304.0,16127.0,9959.0,-0.71,10.33,5.78,16.71,22.0,1.62,1.24,29.56,X40N,BTT,HEALTHCARE
70,SYMPHONY,1306.00,-41.71,41.0,M-SC,18.29,124031.0,-47110.0,47057.0,-0.68,-27.53,37.94,-0.03,196.0,-1.00,0.90,1.68,OX40N,NTT,DURABLES


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.0,105.34,60.0,M-SC,3.96,88471.0,-12306.0,12395.0,-0.21,-12.21,14.01,0.09,245.0,-0.99,0.64,16.23,OX40N,NTT,DURABLES
78,UJJIVANSFB,60.0,97.20,56.0,H-SC,9.54,124254.0,-18225.0,37748.0,0.59,-12.79,30.38,13.70,163.0,-0.48,0.90,46.65,OX40N,NTT,BANKS
45,KANSAINER,340.0,-67.06,63.0,H-SC,1.41,226800.0,-42867.0,79199.0,1.01,-15.90,34.92,13.47,138.0,-0.54,1.64,14.71,XY24,NTT,PAINTS
19,CERA,9475.0,-19.09,52.0,H-SC,2.07,147453.0,-28450.0,70468.0,0.58,-16.17,47.79,23.89,149.0,-0.40,1.06,27.35,OX40N,NTT,CERAMICS
72,TATAELXSI,9161.0,-19.34,55.0,H-MC,7.64,109174.0,-22661.0,64882.0,1.11,-17.19,59.43,32.03,98.0,-0.35,0.79,22.28,OX40N,NTT,IT


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-24.83,60.0,H-LC,12.32,298976.0,-46980.0,121833.0,0.25,-13.58,40.75,21.64,1.0,-0.39,2.16,5.91,X40,ATH,IT
40,INFY,2275.00,-15.65,59.0,H-LC,8.40,329064.0,16176.0,155516.0,1.48,5.17,47.26,54.87,3.0,0.10,2.37,12.16,X40,BTT,IT
42,ITC,452.00,-37.50,51.0,H-LC,2.52,201199.0,1061.0,19838.0,0.53,0.53,9.86,10.44,4.0,0.05,1.45,6.12,X40,NTT,FMCG
83,VBL,671.64,-17.03,39.0,H-LC,6.38,298903.0,-16939.0,129604.0,-0.54,-5.36,43.36,35.67,5.0,-0.13,2.16,7.75,X40N,ATH,FMCG
1,ABB,7934.00,-37.63,68.0,H-LC,10.34,260645.0,-974.0,120183.0,0.82,-0.37,46.11,45.57,7.0,-0.01,1.88,10.91,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,44.35,63.0,M-MC,6.95,227425.0,2463.0,165202.0,-0.51,1.09,72.64,74.53,192.0,0.01,1.64,33.70,XY24,BTT,STEEL
15,BLUESTARCO,2326.38,13.77,65.0,H-MC,3.57,196420.0,31750.0,36220.0,1.24,19.28,18.44,41.28,89.0,0.88,1.42,28.80,X40N,ATH,AC
84,VOLTAS,1918.49,0.32,55.0,H-MC,2.47,210570.0,18828.0,77195.0,-0.57,9.82,36.66,50.08,99.0,0.24,1.52,17.28,XY25,ATH,AC
30,HAVELLS,2069.16,0.06,58.0,H-MC,1.87,248056.0,325.0,74739.0,-0.21,0.13,30.13,30.30,92.0,0.00,1.79,13.44,X40,ATH,ELECTRICAL
18,CAMS,5211.76,-0.07,65.0,H-SC,3.84,113546.0,11542.0,32383.0,1.03,11.32,28.52,43.06,122.0,0.36,0.82,30.37,X40N,ATH,MISC


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,247.44,-43.66,63.0,H-MC,6.80,112508.0,12004.0,18631.0,-0.46,11.94,16.56,30.49,79.0,0.64,0.81,48.22,MH,ATH,METALS
85,WHIRLPOOL,2270.00,-40.41,53.0,M-SC,0.49,100815.0,9316.0,69431.0,0.19,10.18,68.87,86.07,223.0,0.13,0.73,46.40,XR,NTT,DURABLES
37,INDIAMART,4850.92,-51.84,43.0,H-SC,11.68,134540.0,11204.0,122552.0,-0.48,9.08,91.09,108.45,119.0,0.09,0.97,33.47,AR,ATH,MISC
86,WIPRO,420.00,-8.80,62.0,M-LC,5.62,159979.0,9034.0,100419.0,1.53,5.98,62.77,72.51,53.0,0.09,1.15,12.37,XR,NTT,IT
12,BANKINDIA,190.00,-29.19,65.0,H-MC,6.13,183837.0,4029.0,108390.0,-0.09,2.24,58.96,62.52,88.0,0.04,1.33,37.09,XR,NTT,BANKS


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,RAJESHEXPO,518.00,1891.16,67.0,L-SC,3.06,54325.0,-82852.0,82943.0,0.08,-60.40,152.68,0.07,267.0,-1.00,0.39,35.11,OX40N,NTT,JEWELLERY
22,COFFEEDAY,80.00,-43.66,51.0,L-SC,42.31,84806.0,-28743.0,68795.0,-0.36,-25.31,81.12,35.27,268.0,-0.42,0.61,107.57,XR,NTT,HOTELS
3,ALKYLAMINE,4546.37,-4.00,48.0,H-SC,12.75,89298.0,-11665.0,106193.0,-0.82,-11.55,118.92,93.63,148.0,-0.11,0.64,33.01,SR,ATH,CHEMICALS
49,MASFIN,398.61,-16.89,52.0,H-SC,12.99,94500.0,-3480.0,25080.0,-0.43,-3.55,26.54,22.05,152.0,-0.14,0.68,37.36,XR,ATH,FINANCE
50,NATIONALUM,247.44,-43.66,63.0,H-MC,6.80,112508.0,12004.0,18631.0,-0.46,11.94,16.56,30.49,79.0,0.64,0.81,48.22,MH,ATH,METALS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,ATULAUTO,844.0,3991.89,66.0,M-SC,5.77,127523.0,-19845.0,71668.0,3.19,-13.47,56.20,35.16,236.0,-0.28,0.92,30.25,XY24,NTT,AUTO
81,VAIBHAVGBL,521.0,67.49,73.0,H-SC,7.20,145140.0,-37635.0,148710.0,1.64,-20.59,102.46,60.77,125.0,-0.25,1.05,30.00,XR,NTT,JEWELLERY
22,COFFEEDAY,80.0,-43.66,51.0,L-SC,42.31,84806.0,-28743.0,68795.0,-0.36,-25.31,81.12,35.27,268.0,-0.42,0.61,107.57,XR,NTT,HOTELS
75,TITAGARH,1548.0,1.18,69.0,H-SC,3.26,165821.0,-26194.0,105081.0,0.57,-13.64,63.37,41.08,158.0,-0.25,1.20,40.51,XY24,NTT,ENGINEERING
61,SAMMAANCAP,326.0,-171.59,61.0,M-SC,2.61,81888.0,-20322.0,113710.0,1.04,-19.88,138.86,91.37,208.0,-0.18,0.59,32.93,XY25,NTT,FINANCE


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.83
1,25,43.31
2,50,73.76


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.04
LC    32.98
MC    22.94
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.95
X40      15.23
X40N     13.02
XR       11.63
AR        8.66
OX40N     8.15
XY25      8.13
X200      1.80
MH        1.69
X5K       1.42
SR        1.28
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    28.99
H-LC    26.33
H-MC    20.02
M-SC    13.51
M-LC     5.59
M-MC     2.62
L-SC     1.54
L-LC     1.06
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,13.54,-11.97,73.03
FMCG,11.72,-4.30,40.79
FINANCE,9.62,-15.32,60.49
MISC,7.40,-11.95,74.21
BANKS,6.26,-15.23,75.85
PAINTS,5.81,-14.79,31.90
ELECTRICAL,5.68,-4.51,62.11
AC,3.65,4.24,28.19
INSURANCE,3.38,-7.20,43.97


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2834004.0,22
XR,1298630.0,14
AR,1295075.0,9
X40,736499.0,10
X40N,606253.0,10
OX40N,518112.0,10
XY25,396695.0,6
SR,243329.0,2
X5K,120513.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3392387.0,29
M-SC,1374174.0,17
H-LC,1223928.0,15
H-MC,1180226.0,15
M-LC,375190.0,4
M-MC,314502.0,2
L-SC,247872.0,3
L-MC,60506.0,1
L-LC,36922.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1021160.0      6
M-SC       XY24       793912.0      7
H-SC       AR         782288.0      5
           XR         750547.0      7
H-MC       XY24       489573.0      4
H-LC       X40        452227.0      5
           AR         352029.0      2
M-MC       XY24       314502.0      2
H-SC       X40N       285184.0      4
           OX40N      247877.0      4
           SR         243329.0      2
H-LC       X40N       224003.0      4
H-MC       X40        214727.0      4
           XY25       186957.0      2
L-SC       XR         164929.0      2
M-SC       AR         160758.0      2
M-LC       XY24       154258.0      2
M-SC       OX40N      122410.0      4
M-LC       X5K        120513.0      1
M-SC       XR         113839.0      2
           XY25       113710.0      1
H-MC       XR         108390.0      1
M-LC       XR         100419.0      1
H-MC       X40N        97066.0      2
L-SC       OX40N       82943.0      1
H-LC       X200        75964.0      1
M-SC       X40         69545.0      1
H-MC       OX40N       64882.0      1
H-SC       MH          62002.0      1
H-LC       XY24        60599.0      1
L-MC       XR          60506.0      1
H-LC       XY25        59106.0      2
L-LC       XY25        36922.0      1
H-MC       MH          18631.0      1

In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
